## Documnet-Based Question Answer System

 Benjamin Axline

First install necessary libraries.

In [1]:
!pip install --upgrade pip
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils  
!pip install -U langchain-community
!pip install tiktoken -q
!pip install pinecone-client -q
!pip install langchain-openai



zsh:1: no matches found: unstructured[local-inference]
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


#### These are the keys for OpenAI and Pinecone

In [1]:
import os 

# os.environ["OPENAI_API_KEY"] = "YOUR OPENAI KEY"
# os.environ["PINECONE_API_KEY"] = "YOUR PINECONE KEY"

#### Helper Functions

In [3]:
def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

def load_pdf(pdf):
    loader = PyPDFLoader(pdf)
    pages = loader.load_and_split()
    return pages

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

def get_similar_docs(query, k=2, score=False):
    if score:
        similar_docs = index.similarity_search_with_score(query, k=2)
    else:
        similar_docs = index.similarity_search(query, k=2)
    return similar_docs

def get_answer(query):
    similar_docs = get_similar_docs(query)
    answer = chain.run(input_documents=similar_docs, question=query)
    return answer


#### Load and Split documents

In [5]:
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import PyPDFLoader



directory = '/teamspace/studios/this_studio/data'

# Load documents
# documents = load_docs(directory)
documents = load_pdf('data/UserGuide.pdf')

# split documents
docs = split_docs(documents=documents)
print(len(docs))


1772


#### Create embeddings and send to Pinecone

In [ ]:

# embeddings
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# pinecone
pinecone.Pinecone(api_key='dfcf89d1-9a35-4d41-8da8-971c33ac8db2')
index_name = 'question-answer-bot'
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)


#### Initialize the Large Language Model

In [ ]:

# LLM
model_name = 'gpt-4'
llm = ChatOpenAI(model_name=model_name)

chain = load_qa_chain(llm, chain_type='stuff')
# query = input("Your question: ")
# answer = get_answer(query)
# print(f'Question: \n\n{query}\n')
# print(f'Answer: \n\n{answer}\n')


#### Ask question, get answer

In [6]:
query = input("Your question: ")
answer = get_answer(query)
# print(f'Question: \n\n{query}\n')
# print(f'Answer: \n\n{answer}\n')

print('Question:\n')
print(query)
print('\nAnswer:\n')
print(answer)



Question:

How do I install Silhouette?

Answer:

To install Silhouette, follow these steps:

1. Download the Silhouette software at www.borisfx.com.
2. Double-click on the installer file that you downloaded to install Silhouette. The default installation folder on Windows systems is C:\Program Files\BorisFX\Silhouette 2023.5. On Macintosh systems, the default installation folder is /Applications/BorisFX/Silhouette 2023.5.
3. When you get to the Select Components screen, select Silhouette.
4. If you have After Effects installed, you can also select the Silhouette Shape I/O Plug-in.
5. Complete the installation.

To start Silhouette on Windows systems, select Programs > Boris FX Silhouette 2023.5 > Silhouette 2023.5 in the Windows Start menu. 

To start Silhouette on Macintosh systems, go to the /Applications/BorisFX/Silhouette 2023.5 folder and double-click on Silhouette.
